在开发网站的过程中，有一些视图函数虽然处理的对象不同，但是其大致的代码逻辑是一样的。  
Django 把这些相同的逻辑代码抽取了出来，写成了一系列的通用视图函数，即基于类的通用视图（Class Based View）。  

把博客应用中的视图函数改成基于类的通用视图

# ListView

In [ ]:
# blog/views.py

def index(request):
    # ...

def archives(request, year, month):
    # ...

def category(request, pk):
    # ...

## 将 index 视图函数改写为类视图

针对这种从数据库中获取某个模型列表数据（比如这里的 Post 列表）的视图，Django 专门提供了一个 ListView 类视图。

把 index 视图函数改造成类视图函数

In [ ]:
# blog/views.py

from django.views.generic import ListView

class IndexView(ListView):
    model = Post
    template_name = 'blog/index.html'
    context_object_name = 'post_list'

要写一个类视图，首先需要继承 Django 提供的某个类视图。至于继承哪个类视图，需要根据你的视图功能而定。

ListView 就是从数据库中获取某个模型列表数据

通过一些属性来指定这个视图函数需要做的事情。这里我们指定了三个属性
- model。将 model 指定为 Post，告诉 Django 我要获取的模型是 Post。
- template_name。指定这个视图渲染的模板。
- context_object_name。指定获取的模型列表数据保存的变量名。这个变量会被传递给模板。

每一个 URL 对应着一个视图函数,需要将类视图转换成函数视图

In [ ]:
# blog/urls.py

app_name = 'blog'
urlpatterns = [
    url(r'^$', views.index, name='index'),
    ...
]

In [ ]:
# blog/urls.py

app_name = 'blog'
urlpatterns = [
    url(r'^$', views.IndexView.as_view(), name='index'),
    ...
]

## 将 category 视图函数改写为类视图

In [ ]:
# blog/views.py

class CategoryView(ListView):
    model = Post
    template_name = 'blog/index.html'
    context_object_name = 'post_list'

    def get_queryset(self):
        cate = get_object_or_404(Category, pk=self.kwargs.get('pk'))
        return super(CategoryView, self).get_queryset().filter(category=cate)

和 IndexView 不同的地方是，我们覆写了父类的 **get_queryset** 方法。该方法**默认获取指定模型的全部列表数据**。

从 URL 中捕获的分类 id（也就是 pk）获取分类。  
在类视图中，从 URL 捕获的命名组参数值保存在实例的 kwargs 属性（是一个字典）里，非命名组参数值保存在实例的 args 属性（是一个列表）里。所以我们使了 self.kwargs.get('pk') 来获取从 URL 捕获的分类 id 值。  
调用父类的 get_queryset 方法获得全部文章列表，紧接着就对返回的结果调用了 filter 方法来筛选该分类下的全部文章并返回。

我们可以看到 CategoryView 类中指定的属性值和 IndexView 中是一模一样的，所以如果为了进一步节省代码，甚至可以直接继承 IndexView：

In [ ]:
class CategoryView(IndexView):
    def get_queryset(self):
        cate = get_object_or_404(Category, pk=self.kwargs.get('pk'))
        return super(CategoryView, self).get_queryset().filter(category=cate)

In [ ]:
# blog/urls.py

app_name = 'blog'
urlpatterns = [
    ...
    url(r'^category/(?P<pk>[0-9]+)/$', views.CategoryView.as_view(), name='category'),
]

## 将 archives 视图函数改写成类视图

# DetailView

从数据库获取模型的一条记录数据

In [ ]:
# blog/views.py

from django.views.generic import ListView, DetailView

# 记得在顶部导入 DetailView
class PostDetailView(DetailView):
    # 这些属性的含义和 ListView 是一样的
    model = Post
    template_name = 'blog/detail.html'
    context_object_name = 'post'

    def get(self, request, *args, **kwargs):
        # 覆写 get 方法的目的是因为每当文章被访问一次，就得将文章阅读量 +1
        # get 方法返回的是一个 HttpResponse 实例
        # 之所以需要先调用父类的 get 方法，是因为只有当 get 方法被调用后，
        # 才有 self.object 属性，其值为 Post 模型实例，即被访问的文章 post
        response = super(PostDetailView, self).get(request, *args, **kwargs)

        # 将文章阅读量 +1
        # 注意 self.object 的值就是被访问的文章 post
        self.object.increase_views()

        # 视图必须返回一个 HttpResponse 对象
        return response

    def get_object(self, queryset=None):
        # 覆写 get_object 方法的目的是因为需要对 post 的 body 值进行渲染
        post = super(PostDetailView, self).get_object(queryset=None)
        post.body = markdown.markdown(post.body,
                                      extensions=[
                                          'markdown.extensions.extra',
                                          'markdown.extensions.codehilite',
                                          'markdown.extensions.toc',
                                      ])
        return post

    def get_context_data(self, **kwargs):
        # 覆写 get_context_data 的目的是因为除了将 post 传递给模板外（DetailView 已经帮我们完成），
        # 还要把评论表单、post 下的评论列表传递给模板。
        context = super(PostDetailView, self).get_context_data(**kwargs)
        form = CommentForm()
        comment_list = self.object.comment_set.all()
        context.update({
            'form': form,
            'comment_list': comment_list
        })
        return context